<a href="https://colab.research.google.com/github/nguyendinhnien/myplay_analytics_2026/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# library

In [ ]:
import pandas as pd

# helper

In [ ]:
def parse_ab_test(ab_test_str):
    """
    Parse abTest column value into array of dictionaries.

    Args:
        ab_test_str: String with format like "36_A#53_A#54_A#41_B#42_B#60_A#46_B#"

    Returns:
        List of dicts with 'testId' and 'label' keys
        Example: [{'testId': 36, 'label': 'A'}, {'testId': 53, 'label': 'A'}, ...]
    """
    if pd.isna(ab_test_str) or ab_test_str == '':
        return []

    result = []
    tests = ab_test_str.strip('#').split('#')

    for test in tests:
        if test:
            parts = test.split('_')
            if len(parts) == 2:
                result.append({
                    'testId': int(parts[0]),
                    'label': parts[1]
                })

    return result

# Extract label for testId 61
def get_test_label(test_list, test_id):
    for test in test_list:
        if test['testId'] == test_id:
            return test['label']
    return None

# data processing

## files

In [ ]:
df_event = pd.read_csv('data/eventlog_0109_01112_utc.csv')
df_diminfo = pd.concat([
    pd.read_csv('data/diminfo_0106_0108_gtm7.csv'),
    pd.read_csv('data/diminfo_0109_0111_gtm7.csv')
])
df_goldjourney = pd.concat([
        pd.read_csv('data/goldjourney_0106_0108_gtm7.csv'),
        pd.read_csv('data/goldjourney_0109_0111_gtm7.csv')
])

display(df_event.head())
display(df_diminfo.head())
display(df_goldjourney.head())

,__time,abTestCampaign,actionId,age,country,eventAction,eventFullName,eventName,extra_1,extra_2,...,numTransInLife,platform,socialType,state,ticketUsed,totalTicket,totalVndSpendInLife,userId,username,vndSpendInEvent
0,2026-01-09T07:35:48.311Z,36_A#53_A#54_B#41_A#42_C#60_B#61_B#46_C#,mini_game,357,us,login,Magic_Cauldron_2026-01-09,Magic_Cauldron,NaN,NaN,...,130,3,facebook,0,0,6,6226760,1359702567,fb.bnnhqf,0
1,2026-01-09T07:35:48.325Z,36_C#53_B#54_B#41_A#42_B#59_B#61_B#46_B#,mini_game,20,us,login,Magic_Cauldron_2026-01-09,Magic_Cauldron,NaN,NaN,...,0,3,NaN,0,0,6,0,1372762964,fb.bzxwcb,0
2,2026-01-09T07:35:49.513Z,36_B#53_A#54_A#41_A#42_B#60_A#61_B#46_B#,mini_game,97,us,login,Magic_Cauldron_2026-01-09,Magic_Cauldron,NaN,NaN,...,10,3,NaN,0,0,6,1004190,1370422354,270aebcd0b894adfba1e30908565bcaf,0
3,2026-01-09T07:35:49.602Z,36_B#53_B#54_B#41_B#42_B#60_B#61_B#46_C#,mini_game,33,us,login,Magic_Cauldron_2026-01-09,Magic_Cauldron,NaN,NaN,...,0,3,NaN,0,0,6,0,1371179194,gg.115693827641734542703,0
4,2026-01-09T07:35:53.713Z,36_B#53_B#54_A#41_B#42_C#60_A#61_B#46_A#,mini_game,66,us,login,Magic_Cauldron_2026-01-09,Magic_Cauldron,NaN,NaN,...,154,3,NaN,0,0,6,41566692,1371385349,fb.bynuwy,0


,unique_key,log_time,uId,dailyGames,dailyBankrupts,dailyPays,loginCount,isNewUser,kickCount,supportedCount,...,minVip,maxVip,isOldPay,isPayUser,country,lastLoginDayGap,lastPayDayGap,churnPaySegment,churnSegment,isClone
0,efe363cb154387ff4641a0d56b08a16b,2026-01-06,1369440051,4,1,0,1,False,1,1,...,0,0,False,False,us,4,31,31,7,True
1,92ab7870e1a28835ee570664721f02fc,2026-01-06,1372904569,3,1,0,1,False,1,1,...,0,0,False,False,us,11,31,31,15,False
2,e3d3bc23bfe3c9671b29cf1f0123b873,2026-01-06,1366421641,1,1,0,1,False,0,0,...,0,0,False,False,us,6,31,31,7,False
3,0d8e442f811a979c7292515c3e79e5a7,2026-01-06,1360945749,16,3,0,4,False,3,1,...,10,10,True,False,us,1,1,1,1,False
4,fc4f06befaa9f0a4444990fc3adc96dc,2026-01-06,1373286533,21,3,0,2,True,1,1,...,0,0,False,False,us,0,0,0,0,False


,unique_key,log_time,uId,abTest,userAge,os,country,playSegment,paySegment,isOldPay,...,rev,firstBet,betArr,matchCount,kickCount,matchToNextPay,nextPayRev,matchJourneyTh,isChurn1,isChurn7
0,cc5228c014ef58bc3a305b42f13f2040,2026-01-06,1369440051,36_B#53_B#54_A#41_A#42_B#60_C#46_B#,124.0,ios,us,ACTIVE1,G0,False,...,0,100000,"[""100000"", ""100000"", ""100000"", ""100000""]",4,1,0,0,1,0.0,NaN
1,f048e58b37b24f6f2b37a0b6680bd044,2026-01-06,1372904569,36_C#53_A#54_B#41_A#42_A#59_A#46_C#,11.0,ios,us,REACTIVE2,G0,False,...,0,0,[],0,0,3,0,0,1.0,NaN
2,9c9e2d4b4f1882456378375db50ea71f,2026-01-06,1372904569,36_C#53_A#54_B#41_A#42_A#59_A#46_C#,11.0,ios,us,REACTIVE2,G0,False,...,0,600000,"[""600000""]",1,1,2,0,1,1.0,NaN
3,6214638d0010031e3edecf70fa588fb2,2026-01-06,1372904569,36_C#53_A#54_B#41_A#42_A#59_A#46_C#,11.0,ios,us,REACTIVE2,G0,False,...,0,600000,"[""600000"", ""600000""]",2,1,0,0,2,1.0,NaN
4,fdeb1931335a78d7b1392d634bd0071c,2026-01-06,1366421641,36_A#53_A#54_B#41_A#42_A#59_C#46_B#,209.0,ios,us,LAPSE2,G0,False,...,0,0,[],0,0,1,0,0,1.0,NaN


## transform

In [ ]:
# Convert __time from UTC to GMT+7 and extract date for df_event
df_event['date_time'] = pd.to_datetime(df_event['__time'], utc=True).dt.tz_convert('Asia/Bangkok')
df_event['date'] = df_event['date_time'].dt.date

# Convert log_time to date for df_diminfo and df_goldjourney
df_diminfo['date'] = pd.to_datetime(df_diminfo['log_time']).dt.date
df_goldjourney['date'] = pd.to_datetime(df_goldjourney['log_time']).dt.date

# Add period column based on date
cutoff_date = pd.Timestamp('2026-01-08').date()

df_diminfo['period'] = df_diminfo['date'].apply(lambda x: '1_before' if x <= cutoff_date else '2_after')
df_goldjourney['period'] = df_goldjourney['date'].apply(lambda x: '1_before' if x <= cutoff_date else '2_after')

In [ ]:
df_diminfo['abTest_list'] = df_diminfo['ABTest'].apply(parse_ab_test)
df_diminfo['eventTest'] = df_diminfo['abTest_list'].apply(lambda x: get_test_label(x, 61))

In [ ]:
df_goldjourney['abTest_list'] = df_goldjourney['abTest'].apply(parse_ab_test)
df_goldjourney['eventTest'] = df_goldjourney['abTest_list'].apply(lambda x: get_test_label(x, 61))

In [ ]:
df_goldjourney_before = df_goldjourney[(df_goldjourney['period'] == '1_before') & (df_goldjourney['os'] == 'android')]
print(df_goldjourney_before['date'].unique())

# find only 1 transaction during 3 days by uId count appearance of actionType == Purchase
df_gold_journey_purchaseCount = df_goldjourney_before.groupby('uId').apply(
    lambda group: pd.Series({
        'ta_trans': (group['actionType'] == 'Purchase').sum(),
        'ta_firstPurchase': group.loc[group['actionType'] == 'Purchase', 'actionName'].iloc[0] if (group['actionType'] == 'Purchase').any() else 'N/A'
    })
).reset_index()

df_gold_journey_purchaseCount_daily = df_goldjourney_before.groupby(['uId', 'date']).apply(
    lambda group: pd.Series({
        'daily_trans': (group['actionType'] == 'Purchase').sum(),
        'daily_firstPurchase': group.loc[group['actionType'] == 'Purchase', 'actionName'].iloc[0] if (group['actionType'] == 'Purchase').any() else 'N/A'
    })
).reset_index()


# left join df_goldjourney with df_gold_journey_purchaseCount on uId
df_before_merged = pd.merge(df_goldjourney_before, df_gold_journey_purchaseCount, on='uId', how='left')
# left join df_goldjourney with df_gold_journey_purchaseCount on uId, log_time
df_before_merged2 = pd.merge(df_before_merged, df_gold_journey_purchaseCount_daily, on=['uId', 'date'], how='left')

[datetime.date(2026, 1, 6) datetime.date(2026, 1, 7)
 datetime.date(2026, 1, 8)]


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\357815033.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_gold_journey_purchaseCount = df_goldjourney_before.groupby('uId').apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\357815033.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_gold_journey_purchaseCount_daily = df_goldjourney_before.groupby(['uId', 'date']).apply(


In [ ]:
df_goldjourney_after = df_goldjourney[(df_goldjourney['period'] == '2_after') & (df_goldjourney['os'] == 'android')]
print(df_goldjourney_before['date'].unique())

# find only 1 transaction during 3 days by uId count appearance of actionType == Purchase
df_gold_journey_purchaseCount = df_goldjourney_after.groupby('uId').apply(
    lambda group: pd.Series({
        'ta_trans': (group['actionType'] == 'Purchase').sum(),
        'ta_firstPurchase': group.loc[group['actionType'] == 'Purchase', 'actionName'].iloc[0] if (group['actionType'] == 'Purchase').any() else 'N/A'
    })
).reset_index()

df_gold_journey_purchaseCount_daily = df_goldjourney_after.groupby(['uId', 'date']).apply(
    lambda group: pd.Series({
        'daily_trans': (group['actionType'] == 'Purchase').sum(),
        'daily_firstPurchase': group.loc[group['actionType'] == 'Purchase', 'actionName'].iloc[0] if (group['actionType'] == 'Purchase').any() else 'N/A'
    })
).reset_index()


# left join df_goldjourney with df_gold_journey_purchaseCount on uId
df_after_merged = pd.merge(df_goldjourney_after, df_gold_journey_purchaseCount, on='uId', how='left')
# left join df_goldjourney with df_gold_journey_purchaseCount on uId, log_time
df_after_merged2 = pd.merge(df_after_merged, df_gold_journey_purchaseCount_daily, on=['uId', 'date'], how='left')

[datetime.date(2026, 1, 6) datetime.date(2026, 1, 7)
 datetime.date(2026, 1, 8)]


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2839310091.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_gold_journey_purchaseCount = df_goldjourney_after.groupby('uId').apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2839310091.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_gold_journey_purchaseCount_daily = df_goldjourney_after.groupby(['uId', 'date']).apply(


## validate

In [ ]:
# correct label event by check uId has in log event

In [ ]:
# Get set of uIds from event log
event_uids = set(df_event['userId'].unique())

# Function to assign eventTest_2
def assign_event_test_2(row):
    if row['os'] != 'android':
        return None
    elif row['uId'] in event_uids:
        return 'B'
    else:
        return 'A'

# Apply to df_diminfo
df_diminfo['eventTest_2'] = df_diminfo.apply(assign_event_test_2, axis=1)

# Apply to df_goldjourney
df_goldjourney['eventTest_2'] = df_goldjourney.apply(assign_event_test_2, axis=1)

## filter

In [ ]:
df_diminfo_android = df_diminfo[df_diminfo['os'] == 'android']
df_goldjourney_android = df_goldjourney[df_goldjourney['os'] == 'android']

In [ ]:
df_before_ta = df_before_merged2[(df_before_merged2['ta_trans'] == 1) & (df_before_merged2['matchCount'] > 0)]
df_before_ta_low = df_before_ta[df_before_ta['ta_firstPurchase'].isin(['USD 2', 'USD 5', 'USD 10'])]

# analysis

## overall

### KPI metric

In [ ]:
summary_0 = df_diminfo.groupby('date').agg(
    login=('uId', 'nunique'),
    pu=('uId', lambda x: df_diminfo.loc[x.index][df_diminfo.loc[x.index, 'dailyPays'] > 0]['uId'].nunique()),
    trans = ('dailyPays', 'sum'),
    totalRev=('rev', 'sum')
).reset_index()

display(summary_0)

summary_0_android = df_diminfo_android.groupby('date').agg(
    login=('uId', 'nunique'),
    pu=('uId', lambda x: df_diminfo_android.loc[x.index][df_diminfo_android.loc[x.index, 'dailyPays'] > 0]['uId'].nunique()),
    trans = ('dailyPays', 'sum'),
    totalRev=('rev', 'sum')
).reset_index()

display(summary_0_android)

,date,login,pu,trans,totalRev
0,2026-01-06,15831,1934,2254,210404000
1,2026-01-07,15926,1990,2272,218638000
2,2026-01-08,16091,1880,2359,229287000
3,2026-01-09,16244,1931,2603,269790000
4,2026-01-10,15638,1871,2439,246491000
5,2026-01-11,15694,2006,2326,228436000


,date,login,pu,trans,totalRev
0,2026-01-06,4381,428,698,66723000
1,2026-01-07,4318,446,682,59202000
2,2026-01-08,4617,438,762,68701000
3,2026-01-09,4732,447,791,81236000
4,2026-01-10,4313,458,815,78775000
5,2026-01-11,4309,452,731,76981000


In [ ]:
summary_1_android = df_diminfo_android.groupby(['date', 'eventTest']).agg(
    login=('uId', 'nunique'),
    pu=('uId', lambda x: df_diminfo_android.loc[x.index][df_diminfo_android.loc[x.index, 'dailyPays'] > 0]['uId'].nunique()),
    trans = ('dailyPays', 'sum'),
    totalRev=('rev', 'sum')
).reset_index()
display(summary_1_android)

,date,eventTest,login,pu,trans,totalRev
0,2026-01-08,A,962,130,243,20654000
1,2026-01-08,B,980,136,257,26749000
2,2026-01-09,A,2330,232,386,33074000
3,2026-01-09,B,2265,221,364,37099000
4,2026-01-10,A,2129,231,376,39192000
5,2026-01-10,B,2146,244,438,39537000
6,2026-01-11,A,2110,218,326,33166000
7,2026-01-11,B,2162,256,405,43815000


### Event metric

## TA

In [ ]:
# group df_merged by log_time then :
## userCount = count unique uId
## pu = count unique uId where has rev > 0
## pu_1trans1d = count unique uId where has only 1 row rev > 0
## sumRev = sum rev
## pu_1trans3d = count unique uId where ta_trans == 1

ta_before_summary = df_before_merged2.groupby('log_time').apply(
    lambda group: pd.Series({
        'userCount': group['uId'].nunique(),
        'pu': group.loc[group['rev'] > 0, 'uId'].nunique(),
        'pu_1trans1d': group.loc[group['daily_trans'] == 1, 'uId'].nunique(),
        'pu_1trans3d': group.loc[group['ta_trans'] == 1, 'uId'].nunique(),
        'sumRev': group['rev'].sum(),
        'sumRev_1trans1d': group.loc[group['daily_trans'] == 1, 'rev'].sum(),
        'sumRev_1trans3d': group.loc[group['ta_trans'] == 1, 'rev'].sum()
    })
).reset_index()
display(ta_before_summary)

# group df_merged by log_time then :
## userCount = count unique uId
## pu = count unique uId where has rev > 0
## pu_1trans1d = count unique uId where has only 1 row rev > 0
## sumRev = sum rev
## pu_1trans3d = count unique uId where ta_trans == 1

ta_after_summary = df_after_merged2.groupby('log_time').apply(
    lambda group: pd.Series({
        'userCount': group['uId'].nunique(),
        'pu': group.loc[group['rev'] > 0, 'uId'].nunique(),
        'pu_1trans1d': group.loc[group['daily_trans'] == 1, 'uId'].nunique(),
        'pu_1trans3d': group.loc[group['ta_trans'] == 1, 'uId'].nunique(),
        'sumRev': group['rev'].sum(),
        'sumRev_1trans1d': group.loc[group['daily_trans'] == 1, 'rev'].sum(),
        'sumRev_1trans3d': group.loc[group['ta_trans'] == 1, 'rev'].sum()
    })
).reset_index()
display(ta_after_summary)

ta_after_summary2 = df_after_merged2.groupby(['log_time', 'eventTest_2']).apply(
    lambda group: pd.Series({
        'userCount': group['uId'].nunique(),
        'pu': group.loc[group['rev'] > 0, 'uId'].nunique(),
        'pu_1trans1d': group.loc[group['daily_trans'] == 1, 'uId'].nunique(),
        'pu_1trans3d': group.loc[group['ta_trans'] == 1, 'uId'].nunique(),
        'sumRev': group['rev'].sum(),
        'sumRev_1trans1d': group.loc[group['daily_trans'] == 1, 'rev'].sum(),
        'sumRev_1trans3d': group.loc[group['ta_trans'] == 1, 'rev'].sum()
    })
).reset_index()
display(ta_after_summary2)

C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\3627073517.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ta_before_summary = df_before_merged2.groupby('log_time').apply(


,log_time,userCount,pu,pu_1trans1d,pu_1trans3d,sumRev,sumRev_1trans1d,sumRev_1trans3d
0,2026-01-06,4315,359,203,262,69094900,18881000,10874500
1,2026-01-07,4250,366,213,256,61160100,20195500,9321000
2,2026-01-08,4535,384,210,256,70672300,18140100,9129800


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\3627073517.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ta_after_summary = df_after_merged2.groupby('log_time').apply(


,log_time,userCount,pu,pu_1trans1d,pu_1trans3d,sumRev,sumRev_1trans1d,sumRev_1trans3d
0,2026-01-09,4652,397,218,279,84295300,20147700,9464400
1,2026-01-10,4244,408,240,284,81977000,23780500,13073300
2,2026-01-11,4238,379,217,261,80112800,22131400,11161300


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\3627073517.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ta_after_summary2 = df_after_merged2.groupby(['log_time', 'eventTest_2']).apply(


,log_time,eventTest_2,userCount,pu,pu_1trans1d,pu_1trans3d,sumRev,sumRev_1trans1d,sumRev_1trans3d
0,2026-01-09,A,2735,209,110,148,45696800,10611600,5234100
1,2026-01-09,B,1917,188,108,131,38598500,9536100,4230300
2,2026-01-10,A,2130,199,123,142,41012400,12284600,6429100
3,2026-01-10,B,2114,209,117,142,40964600,11495900,6644200
4,2026-01-11,A,2108,175,102,129,34583300,10946200,5258000
5,2026-01-11,B,2130,204,115,132,45529500,11185200,5903300


In [ ]:
df_summary_3d = pd.DataFrame({
    'userCount': [df_before_merged2['uId'].nunique()],
    'pu': [df_before_merged2.loc[df_before_merged2['rev'] > 0, 'uId'].nunique()],
    'pu_1trans3d': [df_before_merged2.loc[df_before_merged2['ta_trans'] == 1, 'uId'].nunique()],
    'pu_1trans3d_low': [df_before_merged2.loc[(df_before_merged2['ta_trans'] == 1) & (df_before_merged2['ta_firstPurchase'].isin(['USD 2', 'USD 5', 'USD 10'])), 'uId'].nunique()],
    'sumRev': [df_before_merged2['rev'].sum()],
    'sumRev_1trans3d': [df_before_merged2.loc[df_before_merged2['ta_trans'] == 1, 'rev'].sum()],
    'sumRev_1trans3d_low': [df_before_merged2.loc[(df_before_merged2['ta_trans'] == 1) & (df_before_merged2['ta_firstPurchase'].isin(['USD 2', 'USD 5', 'USD 10'])), 'rev'].sum()]
})
display(df_summary_3d)

,userCount,pu,pu_1trans3d,pu_1trans3d_low,sumRev,sumRev_1trans3d,sumRev_1trans3d_low
0,6614,719,317,290,200927300,29325300,23183000


In [ ]:
ta_summary = df_before_ta.groupby('ta_firstPurchase').apply(
    lambda group: pd.Series({
        'userCount': group['uId'].nunique(),
        'sumRev': group['rev'].sum(),
    })
).reset_index()
display(ta_summary)

C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\4249284121.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ta_summary = df_before_ta.groupby('ta_firstPurchase').apply(


,ta_firstPurchase,userCount,sumRev
0,USD 10,21,4302000
1,USD 2,200,8986400
2,USD 20,5,1912000
3,USD 3,19,1075500
4,USD 30,1,0
5,USD 5,61,6572500
6,USD 50,1,1195000


In [ ]:
df_tal_play_user_3d = df_before_ta_low.groupby(['uId', 'ta_firstPurchase']).apply(
    lambda group: pd.Series({
        'actionCount': group['matchCount'].count(),
        'matchCount': group['matchCount'].sum(),
        'matchCount_pay': group.loc[group['actionType'] == 'Purchase', 'matchCount'].sum(),
        'matchCount_nonpay': group.loc[group['actionType'] != 'Purchase', 'matchCount'].sum(),
        'payCount': group.loc[group['actionType'] == 'Purchase', 'uId'].count(),
        'payRev': group['rev'].sum()
    })
).reset_index()

df_tal_play_user_3d_summary = df_tal_play_user_3d.groupby(['ta_firstPurchase']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount'].count(),
        'matchCount': round(group['matchCount'].mean(),1),
        'matchCount_pay': round(group['matchCount_pay'].mean(),1)
    })
).reset_index()
display(df_tal_play_user_3d_summary)

df_tal_play_user_3d['temp'] = 'a'
df_tal_play_user_3d_summary = df_tal_play_user_3d.groupby(['temp']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount'].count(),
        'trans': group['payCount'].sum(),
        'pu': group.loc[group['payCount'] >0, 'payCount'].count(),
        'matchCount': round(group['matchCount'].mean(),1),
        'matchCount_pay': round(group['matchCount_pay'].mean(),1),
        'matchCount_nonpay': round(group['matchCount_nonpay'].mean(),1),
        'rev': group.loc[group['payCount'] >0, 'payRev'].sum()
    })
).reset_index()
display(df_tal_play_user_3d_summary)

df_tal_play_journey = df_before_ta_low.groupby(['actionType', 'actionName', 'uId']).apply(
    lambda group: pd.Series({
        'actionCount': group['matchCount'].count(),
        'matchCount': group['matchCount'].sum(),
        'br_count': group['isBankrupt'].sum(),
        'pay_count': group.loc[group['nextPayRev'] >0, 'nextPayRev'].count()
    })
).reset_index()
df_tal_play_journey['matchCount_avg'] = round(df_tal_play_journey['matchCount']/df_tal_play_journey['actionCount'], 1)
df_tal_play_journey_summary = df_tal_play_journey.groupby(['actionType', 'actionName']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount_avg'].count(),
        'matchCount_avg': round(group['matchCount_avg'].mean(),1),
        'matchCount_median': group['matchCount_avg'].median(),
        'action_count': group['actionCount'].sum(),
        'br_count': group['br_count'].sum(),
        'pay_count': group['pay_count'].sum(),
    })
).reset_index()
df_tal_play_journey_summary['brRate'] = round(df_tal_play_journey_summary['br_count']/df_tal_play_journey_summary['action_count'], 2)
df_tal_play_journey_summary['payRate'] = round(df_tal_play_journey_summary['pay_count']/df_tal_play_journey_summary['action_count'], 2)

display(df_tal_play_journey_summary)

C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\791639317.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d = df_before_ta_low.groupby(['uId', 'ta_firstPurchase']).apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\791639317.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_summary = df_tal_play_user_3d.groupby(['ta_firstPurchase']).apply(

,ta_firstPurchase,sample_size,matchCount,matchCount_pay
0,USD 10,21.0,27.7,8.1
1,USD 2,200.0,25.8,9.7
2,USD 5,61.0,24.5,8.5


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\791639317.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_summary = df_tal_play_user_3d.groupby(['temp']).apply(


,temp,sample_size,trans,pu,matchCount,matchCount_pay,matchCount_nonpay,rev
0,a,282.0,261.0,261.0,25.6,9.3,16.3,19860900.0


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\791639317.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_journey = df_before_ta_low.groupby(['actionType', 'actionName', 'uId']).apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\791639317.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_journey_summary = df_tal_play_journey.groupby(['actionType', 'act

,actionType,actionName,sample_size,matchCount_avg,matchCount_median,action_count,br_count,pay_count,brRate,payRate
0,FreeGold,OutOfGoldSupport,260.0,5.7,4.7,570.0,477.0,237.0,0.84,0.42
1,FreeGold,Startup,12.0,8.1,6.5,17.0,14.0,2.0,0.82,0.12
2,Purchase,USD 10,18.0,9.4,4.5,18.0,7.0,0.0,0.39,0.00
3,Purchase,USD 2,188.0,10.3,9.0,188.0,107.0,0.0,0.57,0.00
4,Purchase,USD 5,55.0,9.4,6.0,55.0,32.0,0.0,0.58,0.00
5,RemainGold,login,128.0,5.8,4.4,177.0,92.0,17.0,0.52,0.10


## test - ta

In [ ]:
# Get list of uIds from df_before_ta_low
uids_before_ta_low = df_before_ta_low['uId'].unique()
print(f"Number of uIds in df_before_ta_low: {len(uids_before_ta_low)}")


# Filter df_after_merged2 with these uIds
df_after_merged2_filtered = df_after_merged2[df_after_merged2['uId'].isin(uids_before_ta_low)]
print(f"Number of rows in filtered df_after_merged2: {len(df_after_merged2_filtered)}")
print(f"Number of unique uIds in filtered df_after_merged2: {df_after_merged2_filtered['uId'].nunique()}")

display(df_after_merged2_filtered.head())

ta_low_after = df_after_merged2_filtered[df_after_merged2_filtered['matchCount'] > 0]

# Get list of uIds from B
uids_before_ta_low_B = df_after_merged2_filtered[df_after_merged2_filtered['eventTest_2']=='B']['uId'].unique()
print(f"Number of uIds in df_before_ta_low: {len(uids_before_ta_low_B)}")
df_before_ta_low_filtered_B = df_before_ta_low[df_before_ta_low['uId'].isin(uids_before_ta_low_B)]


Number of uIds in df_before_ta_low: 282
Number of rows in filtered df_after_merged2: 1073
Number of unique uIds in filtered df_after_merged2: 255


,unique_key,log_time,uId,abTest,userAge,os,country,playSegment,paySegment,isOldPay,...,isChurn7,date,period,abTest_list,eventTest,eventTest_2,ta_trans,ta_firstPurchase,daily_trans,daily_firstPurchase
4,3abcd916d303ac564c1e523437cd3422,2026-01-10,1373298901,36_C#53_B#54_B#41_B#42_C#59_B#61_B#46_A#63_A#,4.0,android,us,NEW_USER,G2,False,...,NaN,2026-01-10,2_after,"[{'testId': 36, 'label': 'C'}, {'testId': 53, ...",B,B,2,USD 5,2,USD 5
5,133b0d1fb8e304d1d9b99acf81edf5bd,2026-01-10,1373298901,36_C#53_B#54_B#41_B#42_C#59_B#61_B#46_A#63_A#,4.0,android,us,NEW_USER,G2,False,...,NaN,2026-01-10,2_after,"[{'testId': 36, 'label': 'C'}, {'testId': 53, ...",B,B,2,USD 5,2,USD 5
6,8bdaf86dd69d4426c10a1d14279be9c3,2026-01-10,1373298901,36_C#53_B#54_B#41_B#42_C#59_B#61_B#46_A#63_A#,4.0,android,us,NEW_USER,G2,False,...,NaN,2026-01-10,2_after,"[{'testId': 36, 'label': 'C'}, {'testId': 53, ...",B,B,2,USD 5,2,USD 5
7,a90120c60cf68c9a815847abd3d99639,2026-01-10,1373298901,36_C#53_B#54_B#41_B#42_C#59_B#61_B#46_A#63_A#,4.0,android,us,NEW_USER,G2,False,...,NaN,2026-01-10,2_after,"[{'testId': 36, 'label': 'C'}, {'testId': 53, ...",B,B,2,USD 5,2,USD 5
43,da963a8a631aae3228b157a46ee21208,2026-01-10,1369715991,36_B#53_B#54_A#41_B#42_A#60_A#61_B#46_A#,119.0,android,us,COMMIT,G4,True,...,NaN,2026-01-10,2_after,"[{'testId': 36, 'label': 'B'}, {'testId': 53, ...",B,B,3,USD 2,2,USD 2


Number of uIds in df_before_ta_low: 118


In [ ]:
df_tal_play_user_3d_after = ta_low_after.groupby(['uId', 'eventTest_2']).apply(
    lambda group: pd.Series({
        'actionCount': group['matchCount'].count(),
        'matchCount': group['matchCount'].sum(),
        'matchCount_pay': group.loc[group['actionType'] == 'Purchase', 'matchCount'].sum(),
        'matchCount_nonpay': group.loc[group['actionType'] != 'Purchase', 'matchCount'].sum(),
        'payCount': group.loc[group['actionType'] == 'Purchase', 'uId'].count(),
        'payRev': group['rev'].sum()
    })
).reset_index()

df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['eventTest_2']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount'].count(),
        'trans': group['payCount'].sum(),
        'pu': group.loc[group['payCount'] >0, 'payCount'].count(),
        'matchCount': round(group['matchCount'].mean(),1),
        'matchCount_pay': round(group['matchCount_pay'].mean(),1),
        'matchCount_nonpay': round(group['matchCount_nonpay'].mean(),1),
        'rev': group.loc[group['payCount'] >0, 'payRev'].sum()
    })
).reset_index()
display(df_tal_play_user_3d_summary)
df_tal_play_user_3d_after['temp'] = 'a'
df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['temp']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount'].count(),
        'trans': group['payCount'].sum(),
        'pu': group.loc[group['payCount'] >0, 'payCount'].count(),
        'matchCount': round(group['matchCount'].mean(),1),
        'matchCount_pay': round(group['matchCount_pay'].mean(),1),
        'matchCount_nonpay': round(group['matchCount_nonpay'].mean(),1),
        'rev': group.loc[group['payCount'] >0, 'payRev'].sum()
    })
).reset_index()
display(df_tal_play_user_3d_summary)

df_tal_play_journey = ta_low_after.groupby(['eventTest_2','actionType', 'actionName', 'uId']).apply(
    lambda group: pd.Series({
        'actionCount': group['matchCount'].count(),
        'matchCount': group['matchCount'].sum(),
        'br_count': group['isBankrupt'].sum(),
        'pay_count': group.loc[group['nextPayRev'] >0, 'nextPayRev'].count()
    })
).reset_index()
df_tal_play_journey['matchCount_avg'] = round(df_tal_play_journey['matchCount']/df_tal_play_journey['actionCount'], 1)
df_tal_play_journey_summary = df_tal_play_journey.groupby(['eventTest_2','actionType', 'actionName']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount_avg'].count(),
        'matchCount_avg': round(group['matchCount_avg'].mean(),1),
        'matchCount_median': group['matchCount_avg'].median(),
        'action_count': group['actionCount'].sum(),
        'br_count': group['br_count'].sum(),
        'pay_count': group['pay_count'].sum(),
    })
).reset_index()
df_tal_play_journey_summary['brRate'] = round(df_tal_play_journey_summary['br_count']/df_tal_play_journey_summary['action_count'], 2)
df_tal_play_journey_summary['payRate'] = round(df_tal_play_journey_summary['pay_count']/df_tal_play_journey_summary['action_count'], 2)

display(df_tal_play_journey_summary)

C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2595032471.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_after = ta_low_after.groupby(['uId', 'eventTest_2']).apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2595032471.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['eventTest_2']).apply(

,eventTest_2,sample_size,trans,pu,matchCount,matchCount_pay,matchCount_nonpay,rev
0,A,133.0,151.0,64.0,25.0,9.5,15.5,9966300.0
1,B,112.0,110.0,63.0,26.3,10.6,15.8,8460600.0


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2595032471.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['temp']).apply(


,temp,sample_size,trans,pu,matchCount,matchCount_pay,matchCount_nonpay,rev
0,a,245.0,261.0,127.0,25.6,10.0,15.6,18426900.0


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2595032471.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_journey = ta_low_after.groupby(['eventTest_2','actionType', 'actionName', 'uId']).apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\2595032471.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_journey_summary = df_tal_play_journey.groupby(['event

,eventTest_2,actionType,actionName,sample_size,matchCount_avg,matchCount_median,action_count,br_count,pay_count,brRate,payRate
0,A,FreeGold,OutOfGoldSupport,121.0,5.9,5.00,256.0,207.0,73.0,0.81,0.29
1,A,FreeGold,Startup,2.0,4.5,4.50,6.0,5.0,1.0,0.83,0.17
2,A,Purchase,USD 10,7.0,9.6,9.00,7.0,3.0,1.0,0.43,0.14
3,A,Purchase,USD 2,52.0,9.0,7.75,119.0,82.0,56.0,0.69,0.47
4,A,Purchase,USD 3,6.0,6.5,6.00,8.0,3.0,2.0,0.38,0.25
5,A,Purchase,USD 5,11.0,6.9,5.70,17.0,11.0,3.0,0.65,0.18
6,A,RemainGold,login,59.0,5.7,4.00,82.0,32.0,11.0,0.39,0.13
7,B,FreeGold,OutOfGoldSupport,107.0,4.9,4.70,246.0,201.0,77.0,0.82,0.31
8,B,FreeGold,Startup,2.0,4.5,4.50,2.0,2.0,0.0,1.00,0.00
9,B,Purchase,USD 10,5.0,11.5,7.00,6.0,3.0,1.0,0.50,0.17


In [ ]:
df_tal_play_user_3d_after = df_before_ta_low_filtered_B.groupby(['uId', 'eventTest_2']).apply(
    lambda group: pd.Series({
        'actionCount': group['matchCount'].count(),
        'matchCount': group['matchCount'].sum(),
        'matchCount_pay': group.loc[group['actionType'] == 'Purchase', 'matchCount'].sum(),
        'matchCount_nonpay': group.loc[group['actionType'] != 'Purchase', 'matchCount'].sum(),
        'payCount': group.loc[group['actionType'] == 'Purchase', 'uId'].count(),
        'payRev': group['rev'].sum()
    })
).reset_index()

df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['eventTest_2']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount'].count(),
        'trans': group['payCount'].sum(),
        'pu': group.loc[group['payCount'] >0, 'payCount'].count(),
        'matchCount': round(group['matchCount'].mean(),1),
        'matchCount_pay': round(group['matchCount_pay'].mean(),1),
        'matchCount_nonpay': round(group['matchCount_nonpay'].mean(),1),
        'rev': group.loc[group['payCount'] >0, 'payRev'].sum()
    })
).reset_index()
display(df_tal_play_user_3d_summary)
df_tal_play_user_3d_after['temp'] = 'a'
df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['temp']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount'].count(),
        'trans': group['payCount'].sum(),
        'pu': group.loc[group['payCount'] >0, 'payCount'].count(),
        'matchCount': round(group['matchCount'].mean(),1),
        'matchCount_pay': round(group['matchCount_pay'].mean(),1),
        'matchCount_nonpay': round(group['matchCount_nonpay'].mean(),1),
        'rev': group.loc[group['payCount'] >0, 'payRev'].sum()
    })
).reset_index()
display(df_tal_play_user_3d_summary)

df_tal_play_journey = df_before_ta_low_filtered_B.groupby(['eventTest_2','actionType', 'actionName', 'uId']).apply(
    lambda group: pd.Series({
        'actionCount': group['matchCount'].count(),
        'matchCount': group['matchCount'].sum(),
        'br_count': group['isBankrupt'].sum(),
        'pay_count': group.loc[group['nextPayRev'] >0, 'nextPayRev'].count()
    })
).reset_index()
df_tal_play_journey['matchCount_avg'] = round(df_tal_play_journey['matchCount']/df_tal_play_journey['actionCount'], 1)
df_tal_play_journey_summary = df_tal_play_journey.groupby(['eventTest_2','actionType', 'actionName']).apply(
    lambda group: pd.Series({
        'sample_size': group['matchCount_avg'].count(),
        'matchCount_avg': round(group['matchCount_avg'].mean(),1),
        'matchCount_median': group['matchCount_avg'].median(),
        'action_count': group['actionCount'].sum(),
        'br_count': group['br_count'].sum(),
        'pay_count': group['pay_count'].sum(),
    })
).reset_index()
df_tal_play_journey_summary['brRate'] = round(df_tal_play_journey_summary['br_count']/df_tal_play_journey_summary['action_count'], 2)
df_tal_play_journey_summary['payRate'] = round(df_tal_play_journey_summary['pay_count']/df_tal_play_journey_summary['action_count'], 2)

display(df_tal_play_journey_summary)

C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\1243523152.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_after = df_before_ta_low_filtered_B.groupby(['uId', 'eventTest_2']).apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\1243523152.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['eventT

,eventTest_2,sample_size,trans,pu,matchCount,matchCount_pay,matchCount_nonpay,rev
0,B,118.0,108.0,108.0,25.8,8.4,17.4,8221600.0


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\1243523152.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_user_3d_summary = df_tal_play_user_3d_after.groupby(['temp']).apply(


,temp,sample_size,trans,pu,matchCount,matchCount_pay,matchCount_nonpay,rev
0,a,118.0,108.0,108.0,25.8,8.4,17.4,8221600.0


C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\1243523152.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_journey = df_before_ta_low_filtered_B.groupby(['eventTest_2','actionType', 'actionName', 'uId']).apply(
C:\Users\CPU60303-Local\AppData\Local\Temp\ipykernel_8232\1243523152.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tal_play_journey_summary = df_tal_play_journey.

,eventTest_2,actionType,actionName,sample_size,matchCount_avg,matchCount_median,action_count,br_count,pay_count,brRate,payRate
0,B,FreeGold,OutOfGoldSupport,110.0,5.7,5.0,246.0,217.0,99.0,0.88,0.40
1,B,FreeGold,Startup,7.0,9.8,8.0,12.0,9.0,1.0,0.75,0.08
2,B,Purchase,USD 10,7.0,7.9,3.0,7.0,4.0,0.0,0.57,0.00
3,B,Purchase,USD 2,77.0,9.5,8.0,77.0,43.0,0.0,0.56,0.00
4,B,Purchase,USD 5,24.0,8.2,6.0,24.0,11.0,0.0,0.46,0.00
5,B,RemainGold,login,59.0,5.7,4.0,80.0,45.0,5.0,0.56,0.06
